In [ ]:
%%capture
! pip install langchain-core langgraph>0.2.27
! pip install anthropic
! pip install langchain[anthropic]
! pip install wikipedia
! pip install mysql-connector-python
! pip install pydantic_extra_types phonenumbers pydantic[email]

In [ ]:
%%capture
! apt-get -y install mysql-server
! service mysql start
! mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'mkdirTenorio';FLUSH PRIVILEGES;"

https://drlee.io/mysql-in-google-colab-a-seamless-integration-ab4bd47c419d

https://www.datacamp.com/tutorial/mysql-python

In [ ]:
# Setting Up MySQL on Google Colab (Terminal)

# apt-get -y install mysql-server
# service mysql start

# Secure Your MySQL Installation. Here, we’re setting the root password as ‘root’.
# mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'mkdirTenorio';FLUSH PRIVILEGES;"

# If I want to connect to MySQL Server via its cli
# mysql -u root -p


# Use this command after insert, update, delete commands
# conn.commit()

In [ ]:
import getpass
import os



## Access the database with python

In [ ]:
import mysql.connector
import pandas as pd

# Functions to simplify connection with DB API and print table

def create_connection():
  return mysql.connector.connect(user='root', password='mkdirTenorio', host='localhost',database="contact_db")


In [ ]:
# Create a cursor to interact with the MySQL server
conn = mysql.connector.connect(user='root', password='mkdirTenorio', host='localhost')
cursor = conn.cursor()

# Create a new database named 'contact_db' with cursor.execute()
cursor.execute("CREATE DATABASE IF NOT EXISTS contact_db")

# Switch to the 'library' database
cursor.execute("USE contact_db")

# To drop an existing table use:
# DROP TABLE table_name;
# cursor.execute("DROP TABLE contacts")

# Create the table
cursor.execute('''
    CREATE TABLE contacts (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    nickname VARCHAR(255) NULL,
    phone_number VARCHAR(15) NOT NULL UNIQUE,
    email VARCHAR(255) NULL UNIQUE)
''')

# Always remember to close the cursor and connection when done
cursor.close()
conn.close()

In [ ]:
# SHOW DATABASES;
# SHOW TABLES;
# DESCRIBE contacts;

Insert some examples in the table

In [ ]:
# Create a cursor to interact with the MySQL server
conn = create_connection()
cursor = conn.cursor()

# Insert Sample Data
cursor.execute(
    """
   INSERT INTO contacts (name, nickname, phone_number, email) VALUES
   ('Ana Maria Braga', 'MaisVC', '+12124567890', 'anamaria@globo.com'),
   ('Fausto Silva', 'Faustao', '+4530770800', 'fausto@globo.com'),
   ('Charlie Brown Jr', NULL, '+4531230899', 'charlie@shaike.com'),
   ('Thammy Gretchen', 'Carluxa', '+13105867367', 'tammy@example.com'),
   ('Carlos Bolsonaro', 'Carluxo', '+4538770890', NULL)
    """
)

# Use this command after insert, update, delete commands
conn.commit()

# Always remember to close the cursor and connection when done
cursor.close()
conn.close()

In [ ]:
from pydantic import BaseModel, Field, EmailStr, root_validator, validator
from pydantic_extra_types.phone_numbers import PhoneNumber
from typing import Any

# Schema definition

PhoneNumber.phone_format = 'E164' #'INTERNATIONAL', 'NATIONAL'

class ContactInfoSchema(BaseModel):
    id: int = Field(..., description="Unique id number of the contact.") # required
    name: str = Field(..., description="Name of the contact.") # required
    nickname: str | None = Field(description="Nickname of the contact.", default=None) # optional param
    phone_number: PhoneNumber = Field(..., description="Phone number of the contact.")# required
    email: EmailStr | None = Field(description="Email of the contact.", default=None) # optional param

# Validator used to check, add, or remove a contact.
class CheckContactInput(BaseModel):
    name: str | None = Field(description="Name of the contact.", default=None)
    nickname: str | None = Field(description="Nickname of the contact.", default=None)
    phone_number: PhoneNumber | None = Field(description="Phone number.", default=None)
    email: EmailStr | None = Field(description="Email of the contact.", default=None)


class AddContactInput(BaseModel):
    name: str = Field(description="Name of the contact (required).")
    phone_number: PhoneNumber = Field(description="Phone number of the contact (required).")
    nickname: str | None = Field(description="Optional nickname for the contact.", default=None)
    email: EmailStr | None = Field(description="Optional email address for the contact.", default=None)


class RemoveContactInput(BaseModel):
    name: str | None = Field(description="Name of the contact to remove.", default=None)
    phone_number: PhoneNumber | None = Field(description="Phone number of the contact to remove.", default=None)
    nickname: str | None = Field(description="Nickname of the contact to remove.", default=None)
    email: EmailStr | None = Field(description="Email of the contact to remove.", default=None)

    @root_validator(pre=True) # ensures that the user provides at least one valid field when creating an instance of the RemoveContactInput model.
    def at_least_one_field(cls, values): # values is a dictionary passed to the model, ex: {"name": "John Doe", "phone_number": None, "nickname": None, "email": None}
      if not any(values.values()): # values.values() -> dict_values(['John Doe', None, None, None]); any() -> True if at least one value is not None or empty string.
          raise ValueError("At least one search parameter (name, phone_number, nickname, or email) must be provided")
      return values

class EditContactInput(BaseModel):
    current_name: str = Field(description="Current name of the contact to edit.")
    new_name: str | None = Field(description="New name for the contact.", default=None)
    nickname: str | None = Field(description="New nickname for the contact.", default=None)
    phone_number: PhoneNumber | None = Field(description="New phone number for the contact.", default=None)
    email: EmailStr | None = Field(description="New email for the contact.", default=None)


<ipython-input-9-fddb406e7a12>:37: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @root_validator(pre=True) # ensures that the user provides at least one valid field when creating an instance of the RemoveContactInput model.


In [ ]:
from langchain.agents import tool

# Functions definition

@tool(args_schema=CheckContactInput)
def check_contact(name: str = None, nickname: str = None, phone_number: PhoneNumber = None, email: EmailStr = None)->dict:
    """Use this function when you want to check the existance of a contact. This function dos not alter any contact information.
    Trigger words: CHECK, VERIFY, SEARCH, SEE IF I HAVE A CONTACT.

    Required: at least one of the fields (name, nickname, phone_number, or email)

    Returns a dictionary with success status and message.
    """
    # Verify at least one field was provided
    if not any([name, nickname, phone_number, email]):
        return {"success": False, "message": "No contact information provided. Please specify at least one field to search for."}

    # Update contact details
    info = []
    values = []
    if name:
        info.append("name = %s")
        values.append(name)
    if nickname:
        info.append("nickname = %s")
        values.append(nickname)
    if phone_number:
        info.append("phone_number = %s")
        values.append(phone_number)
    if email:
        info.append("email = %s")
        values.append(email)

    conn = create_connection()
    try:
      check_query = f"SELECT * FROM contacts WHERE {' OR '.join(info)} "
      with conn.cursor(dictionary=True) as cursor: # Set dictionary=True to fetch data as dict
          cursor.execute(check_query, tuple(values))
          result = cursor.fetchall()
      if result:
          # check output against ContactInfoSchema and return a dict
          contact = ContactInfoSchema(**result[0])
          contact = contact.model_dump()
          output = {"success": True, "message": contact}
          return output
      else:
          return  {"success": False, "message": "You do not have a contact with the provided information."}

    except Exception as e:
        # Handle exceptions
        if conn:
            conn.rollback()
        return {"success": False, "message": f"Error checking for the contact: {str(e)}"}
    finally:
        if conn:
            conn.close()


In [ ]:
def print_contacts() -> None:
  """Function used to show/print the contacts list. This function is called every time a user asks to show the contacts"""

  conn = create_connection()
  with conn.cursor(dictionary=True) as cursor: # Set dictionary=True to fetch data as dict
    cursor.execute("SELECT * FROM contacts")

    records = cursor.fetchall()

    df = pd.DataFrame.from_dict(records)

    print(df)


In [ ]:
@tool(args_schema=AddContactInput)
def add_contact(name: str, phone_number: PhoneNumber = None, nickname: str = None, email: EmailStr = None) -> dict:
    """
    Add or create a new contact to the database.

    Use this function to create a new contact record. The contact will be checked to ensure it doesn't already exist.
    Trigger words: ADD, CREATE, NEW

    Required parameters:
    - name: The contact's full name
    - phone_number: A valid phone number

    Optional parameters:
    - nickname: An informal name for the contact
    - email: The contact's email address

    Returns a dictionary with success status and message.
    """
    # Verify at least one search parameter was provided
    if not name or not phone_number:
      return {"success": False, "message": "Both name and phone number are required to add a contact."}

    #Verify if the contact exists.
    contact_info_dict = CheckContactInput(name=name, nickname=nickname, phone_number=phone_number, email=email).model_dump()
    cont_ckt = check_contact(contact_info_dict)

    conn = create_connection()
    try:
      if cont_ckt.get("success"):
        return {"success": False, "message": "Contact already exists"}
      else: # Add new contact
        with conn.cursor() as cursor:
          cursor.execute(
            "INSERT INTO contacts (name, nickname, phone_number, email) VALUES (%s, %s, %s, %s)",
            (name, nickname, phone_number, email)
          )
          conn.commit()
        return {"success": True, "message": "A new contact was added successfully."}
    except Exception as e:
        # Handle exceptions
        if conn:
            conn.rollback()
        return {"success": False, "message": f"Error checking for the contact: {str(e)}"}
    finally:
        if conn:
            conn.close()


@tool(args_schema=RemoveContactInput)
def remove_contact(name: str = None, nickname: str = None, phone_number: PhoneNumber = None, email: EmailStr = None) -> dict:
    """
    Remove or delete an existing contact from the database.

    Use this function to permanently delete a contact record. The contact will be identified by at least one field.
    Trigger words: REMOVE, DELETE, ERASE

    At least one of these parameters must be provided:
    - name: The contact's full name
    - nickname: The contact's nickname
    - phone_number: The contact's phone number
    - email: The contact's email address

    Returns a dictionary with success status and message.
    """
    # Verify at least one search parameter was provided
    if not any([name, nickname, phone_number, email]):
        return {"success": False, "message": "At least one search parameter (name, phone_number, nickname, or email) must be provided."}

    # Verify if contact exists
    contact_info_dict = CheckContactInput(name=name, nickname=nickname, phone_number=phone_number, email=email).model_dump()
    cont_ckt = check_contact(contact_info_dict)

    conn = create_connection()
    try:
      if not cont_ckt.get("success"):
        return {"success": False, "message": "Contact does not exist."}
      else: # remove the contact
        contact_index = cont_ckt["message"].get("id")
        with conn.cursor() as cursor:
          cursor.execute(
            "DELETE FROM contacts WHERE id=%s",
            (contact_index,)
          )
          conn.commit()
        return {"success": True, "message": "contact removed successfully."}
    except Exception as e:
        # Handle exceptions
        if conn:
            conn.rollback()
        return {"success": False, "message": f"Error checking for the contact: {str(e)}"}
    finally:
        if conn:
            conn.close()


In [ ]:
@tool(args_schema=EditContactInput)
def edit_contact(current_name: str, new_name: str = None, nickname: str = None,
                phone_number: PhoneNumber = None, email: EmailStr = None) -> dict:
    """
    Edit an existing contact. This function updates information for a contact identified by their current name.

    Use this function when you need to change contact details like name, nickname, phone number, or email.
    Trigger words: CHANGE, EDIT, MODIFY, UPDATE

    Required: current_name and at least one field to update (new_name, nickname, phone_number, or email)

    Returns a dictionary with success status and message.
    """
    # Verify at least one update field was provided
    if not any([new_name, nickname, phone_number, email]):
        return {"success": False, "message": "No new contact information provided. Please specify at least one field to update."}

    # Check if contact exists
    conn = create_connection()
    try:
        with conn.cursor(dictionary=True) as cursor:
            cursor.execute("SELECT id FROM contacts WHERE name = %s", (current_name,))
            contact = cursor.fetchone()

            if not contact:
                return {"success": False, "message": f"Contact '{current_name}' does not exist."}

            contact_id = contact["id"]

            # Build update query
            updates = []
            values = []
            if new_name:
                updates.append("name = %s")
                values.append(new_name)
            if nickname:
                updates.append("nickname = %s")
                values.append(nickname)
            if phone_number:
                updates.append("phone_number = %s")
                values.append(phone_number)
            if email:
                updates.append("email = %s")
                values.append(email)

            # Add contact_id for the WHERE clause
            values.append(contact_id)

            # Execute update
            update_query = f"UPDATE contacts SET {', '.join(updates)} WHERE id = %s"
            cursor.execute(update_query, tuple(values))
            conn.commit()

            return {
                "success": True,
                "message": f"Contact '{current_name}' updated successfully.",
                "updated_fields": len(updates)
            }
    except Exception as e:
        # Handle exceptions
        if conn:
            conn.rollback()
        return {"success": False, "message": f"Error updating contact: {str(e)}"}
    finally:
        if conn:
            conn.close()

## Edit some contacts

In [ ]:
print_contacts()

   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   3  Charlie Brown Jr     None   +4531230899  charlie@shaike.com
3   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
4   5  Carlos Bolsonaro  Carluxo   +4538770890                None


start with a check. Let us check if the contact with nickname Ali exist.

In [ ]:
contact_check = {"nickname":"MaisVC"}
CheckContactInput(**contact_check)

CheckContactInput(name=None, nickname='MaisVC', phone_number=None, email=None)

In [ ]:
cont_ckt = check_contact(contact_check)
print(cont_ckt,type(cont_ckt))

<ipython-input-16-9e771af0ac86>:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  cont_ckt = check_contact(contact_check)


{'success': True, 'message': {'id': 1, 'name': 'Ana Maria Braga', 'nickname': 'MaisVC', 'phone_number': '+12124567890', 'email': 'anamaria@globo.com'}} <class 'dict'>


In [ ]:
contact_check = {}
cont_ckt = check_contact(contact_check)
print(cont_ckt,type(cont_ckt))

{'success': False, 'message': 'No contact information provided. Please specify at least one field to search for.'} <class 'dict'>


In [ ]:
print(check_contact.name)
print(check_contact.description)
print(check_contact.args)

check_contact
Use this function when you want to check the existance of a contact. This function dos not alter any contact information.
    Trigger words: CHECK, VERIFY, SEARCH, SEE IF I HAVE A CONTACT.

    Required: at least one of the fields (name, nickname, phone_number, or email)

    Returns a dictionary with success status and message.
{'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Name of the contact.', 'title': 'Name'}, 'nickname': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Nickname of the contact.', 'title': 'Nickname'}, 'phone_number': {'anyOf': [{'format': 'phone', 'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Phone number.', 'title': 'Phone Number'}, 'email': {'anyOf': [{'format': 'email', 'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Email of the contact.', 'title': 'Email'}}


## Add, remove, edit a few contacts

In [ ]:
print_contacts()

   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   3  Charlie Brown Jr     None   +4531230899  charlie@shaike.com
3   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
4   5  Carlos Bolsonaro  Carluxo   +4538770890                None


Let us add a new contact Ronaldo and Rony

In [ ]:
# Adding a new contact

# Prepare the input dictionary matching the args_schema
contact_info = AddContactInput(name="Ronaldo", nickname="R9", phone_number="+15575867367").model_dump()
print(contact_info)

# Call the add_contact tool with the input dictionary
add_contact(contact_info)

{'name': 'Ronaldo', 'phone_number': '+15575867367', 'nickname': 'R9', 'email': None}


{'success': True, 'message': 'A new contact was added successfully.'}

In [ ]:
# Prepare the input dictionary matching the args_schema
contact_info = AddContactInput(name="Rony Rustico", nickname="Rony", phone_number="+15574067671").model_dump()
print(contact_info)

# Call the add_contact tool with the input dictionary
add_contact(contact_info)

{'name': 'Rony Rustico', 'phone_number': '+15574067671', 'nickname': 'Rony', 'email': None}


{'success': True, 'message': 'A new contact was added successfully.'}

In [ ]:
print_contacts()

   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   3  Charlie Brown Jr     None   +4531230899  charlie@shaike.com
3   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
4   5  Carlos Bolsonaro  Carluxo   +4538770890                None
5   6           Ronaldo       R9  +15575867367                None
6   7      Rony Rustico     Rony  +15574067671                None


Let us now change the nickname of Rony and add an email.

In [ ]:
# Edit an existing contact

contact_info = EditContactInput(current_name='Rony Rustico', nickname="Rony", email="rustico@uol.com.br").model_dump()
print(contact_info)

{'current_name': 'Rony Rustico', 'new_name': None, 'nickname': 'Rony', 'phone_number': None, 'email': 'rustico@uol.com.br'}


In [ ]:
# Editing a contact
edit_contact(contact_info)

print_contacts()

   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   3  Charlie Brown Jr     None   +4531230899  charlie@shaike.com
3   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
4   5  Carlos Bolsonaro  Carluxo   +4538770890                None
5   6           Ronaldo       R9  +15575867367                None
6   7      Rony Rustico     Rony  +15574067671  rustico@uol.com.br


Removing a existing contact. Now remove Rony Rustico

In [ ]:
# Removing a existing contact

# Prepare the input dictionary matching the args_schema
contact_info = RemoveContactInput(nickname="Rony").model_dump()
print(contact_info)


{'name': None, 'phone_number': None, 'nickname': 'Rony', 'email': None}


In [ ]:
# Call the add_contact tool with the input dictionary
print(remove_contact(contact_info))

print_contacts()

{'success': True, 'message': 'contact removed successfully.'}
   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   3  Charlie Brown Jr     None   +4531230899  charlie@shaike.com
3   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
4   5  Carlos Bolsonaro  Carluxo   +4538770890                None
5   6           Ronaldo       R9  +15575867367                None


---

# Let us create the LLM agents! 🤖 🚀

In [ ]:
import anthropic

# Initialize the Claude client. It initializes a new client object from the Anthropic Python SDK
client = anthropic.Anthropic()
model = "claude-3-7-sonnet-20250219"

In [ ]:
from langchain_anthropic.chat_models import convert_to_anthropic_tool

tools_box = [check_contact,add_contact,remove_contact,edit_contact]
tools = [convert_to_anthropic_tool(t) for t in tools_box]

#tools.append(print_contacts_tool)

tools

[{'name': 'check_contact',
  'input_schema': {'properties': {'name': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None,
     'description': 'Name of the contact.'},
    'nickname': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': 'Nickname of the contact.'},
    'phone_number': {'anyOf': [{'format': 'phone', 'type': 'string'},
      {'type': 'null'}],
     'default': None,
     'description': 'Phone number.'},
    'email': {'anyOf': [{'format': 'email', 'type': 'string'},
      {'type': 'null'}],
     'default': None,
     'description': 'Email of the contact.'}},
   'type': 'object'},
  'description': 'Use this function when you want to check the existance of a contact. This function dos not alter any contact information.\n    Trigger words: CHECK, VERIFY, SEARCH, SEE IF I HAVE A CONTACT.\n\n    Required: at least one of the fields (name, nickname, phone_number, or email)\n\n    Returns a dictionary with success sta

In [ ]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model=model,
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    # api_key="...",
    # base_url="...",
    # other params...
)

chatmodel = llm.bind(tools=tools)

In [ ]:
from langchain_anthropic.output_parsers import ToolsOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy personal assistant. You must be able to excecute tasks related to managing contact information for the user such as verify, create, delete, or edit a contact information like phone number, email, etc."),
    ("user", "{input}"),
])

chain = prompt | chatmodel | ToolsOutputParser() # you can use args_only=True to return only the args, or pydantic_schemas

In [ ]:
message = chain.invoke({"input": "Hi! Do I have a conact named Charlie Brown?"})

print(message)

[{'name': 'check_contact', 'args': {'name': 'Charlie Brown'}, 'id': 'toolu_01Ah5LwhijpKEYWZczwJyT3V', 'type': 'tool_call', 'index': 1}]


In [ ]:
message2 = chain.invoke({"input": "Hi! Please delete the contact named Charlie Brown."})

print(message2)

[{'name': 'remove_contact', 'args': {'name': 'Charlie Brown'}, 'id': 'toolu_018mg3BYkXGE1RJvtogZhpBX', 'type': 'tool_call', 'index': 1}]


In [ ]:
message3 = chain.invoke({"input": "Hi! Please modify the email of the contact named Charlie Brown to charlie@ig.com.br."})

print(message3)

[{'name': 'edit_contact', 'args': {'current_name': 'Charlie Brown', 'email': 'charlie@ig.com.br'}, 'id': 'toolu_011yUw6x6YtrUpUnvwbKuW3d', 'type': 'tool_call', 'index': 1}]


In [ ]:
message = chain.invoke({"input": "Hi!"})

print(message)

[]


In [ ]:
def route(result):
    if not result:
        return print("No tools selected")
    else:
        tools = {
            "check_contact": check_contact,
            "add_contact": add_contact,
            "remove_contact": remove_contact,
            "edit_contact": edit_contact,
        }
        return tools[result[0]["name"]].run(result[0]["args"])

In [ ]:
chain_route = prompt | chatmodel | ToolsOutputParser() | route

In [ ]:
message = chain_route.invoke({"input": "Hi!"})

print(message)

No tools selected
None


In [ ]:
message1 = chain_route.invoke({"input": "Hi! Do I have a conact named Fausto Silva?"})

print(message1)

{'success': True, 'message': {'id': 2, 'name': 'Fausto Silva', 'nickname': 'Faustao', 'phone_number': '+4530770800', 'email': 'fausto@globo.com'}}


In [ ]:
message3 = chain_route.invoke({"input": "please create a contact Charlie Brown Jr with nickname Chuck and phone number +4531230899"})

print(message3)

{'success': False, 'message': 'Contact already exists'}


In [ ]:
print_contacts()

   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   3  Charlie Brown Jr     None   +4531230899  charlie@shaike.com
3   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
4   5  Carlos Bolsonaro  Carluxo   +4538770890                None
5   6           Ronaldo       R9  +15575867367                None


In [ ]:
message13 = chain_route.invoke({"input": "remove the conact named Charlie Brown Jr"})

print(message13)

{'success': True, 'message': 'contact removed successfully.'}


In [ ]:
print_contacts()

   id              name nickname  phone_number               email
0   1   Ana Maria Braga   MaisVC  +12124567890  anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800    fausto@globo.com
2   4   Thammy Gretchen  Carluxa  +13105867367   tammy@example.com
3   5  Carlos Bolsonaro  Carluxo   +4538770890                None
4   6           Ronaldo       R9  +15575867367                None


Test some contact editing.

In [ ]:
message13 = chain_route.invoke({"input": "create a contact named Rony Rustico, nickname Rony, phone number +4538711490"})

print(message13)

{'success': True, 'message': 'A new contact was added successfully.'}


In [ ]:
message5 = chain_route.invoke({"input": "modify the email of the contact named Rony Rustico to rony.rustico@uol.com"})

print(message5)

{'success': True, 'message': "Contact 'Rony Rustico' updated successfully.", 'updated_fields': 1}


In [ ]:
print_contacts()

   id              name nickname  phone_number                 email
0   1   Ana Maria Braga   MaisVC  +12124567890    anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800      fausto@globo.com
2   4   Thammy Gretchen  Carluxa  +13105867367     tammy@example.com
3   5  Carlos Bolsonaro  Carluxo   +4538770890                  None
4   6           Ronaldo       R9  +15575867367                  None
5   8      Rony Rustico     Rony   +4538711490  rony.rustico@uol.com


In [ ]:
message7 = chain_route.invoke({"input": "modify the email of the contact name Ronaldo fifathebest@uol.com"})

print(message7)

print_contacts()

{'success': True, 'message': "Contact 'Ronaldo' updated successfully.", 'updated_fields': 1}
   id              name nickname  phone_number                 email
0   1   Ana Maria Braga   MaisVC  +12124567890    anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800      fausto@globo.com
2   4   Thammy Gretchen  Carluxa  +13105867367     tammy@example.com
3   5  Carlos Bolsonaro  Carluxo   +4538770890                  None
4   6           Ronaldo       R9  +15575867367   fifathebest@uol.com
5   8      Rony Rustico     Rony   +4538711490  rony.rustico@uol.com


# ✅ Now create a conversational chatbot with memory. 🧠 🤖

In this section I will also replace `ToolsOutputParser` by `ToolsAgentOutputParser` for better handling the chain response.

In [ ]:
from langchain_anthropic.output_parsers import ToolsOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant named Wesley Safadão. our task is to manage contact information for the user such as verify, create, delete, or edit a contact informations like contacts name, phone number, email, etc. Keep your answers short."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

from langchain.agents.output_parsers.tools import ToolsAgentOutputParser # instead of simpler ToolsOutputParser()
chain = prompt | chatmodel | ToolsAgentOutputParser()


In [ ]:
result1 = chain.invoke({
    "input": "do I have a contact named Ronaldo?",
    "agent_scratchpad": []
})

result1

[ToolAgentAction(tool='check_contact', tool_input={'name': 'Ronaldo'}, log="\nInvoking: `check_contact` with `{'name': 'Ronaldo'}`\nresponded: [{'text': 'Let me check if you have a contact named Ronaldo in your database.', 'type': 'text'}, {'id': 'toolu_01KE8FAySN3z8Qft7cRzmgLX', 'input': {'name': 'Ronaldo'}, 'name': 'check_contact', 'type': 'tool_use'}]\n\n", message_log=[AIMessage(content=[{'text': 'Let me check if you have a contact named Ronaldo in your database.', 'type': 'text'}, {'id': 'toolu_01KE8FAySN3z8Qft7cRzmgLX', 'input': {'name': 'Ronaldo'}, 'name': 'check_contact', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01DQXiceo7XuHPffU3asGNAL', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1614, 'output_tokens': 72, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--f740ec9f-78d1-47e1-b7f2-c8

In [ ]:
type(result1),type(result1[0])

(list, langchain.agents.output_parsers.tools.ToolAgentAction)

Notice 👀


✅ `ToolsOutputParser()` is fetched as a **dictionary** with the tool_calls key ✅

✅ `ToolsAgentOutputParser()` returns an instance of `ToolAgentAction` or `AgentFinish` with easier to fetch attributes ✅


In [ ]:
result = result1[0]

print(result.tool, "\n")
print(result.tool_input, "\n")
print(result.type, "\n")
print(result.message_log, "\n")
print("what is was before fetched by ToolsOutputParser()", "\n", result.message_log[0].tool_calls)

check_contact 

{'name': 'Ronaldo'} 

AgentActionMessageLog 

[AIMessage(content=[{'text': 'Let me check if you have a contact named Ronaldo in your database.', 'type': 'text'}, {'id': 'toolu_01KE8FAySN3z8Qft7cRzmgLX', 'input': {'name': 'Ronaldo'}, 'name': 'check_contact', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01DQXiceo7XuHPffU3asGNAL', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1614, 'output_tokens': 72, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--f740ec9f-78d1-47e1-b7f2-c86423a3e520-0', tool_calls=[{'name': 'check_contact', 'args': {'name': 'Ronaldo'}, 'id': 'toolu_01KE8FAySN3z8Qft7cRzmgLX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1614, 'output_tokens': 72, 'total_tokens': 1686, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})] 

what is was before f

In [ ]:
result2 = chain.invoke({
    "input": "Hi!",
    "agent_scratchpad": []
})

from langchain.schema.agent import AgentFinish

print(isinstance(result2,AgentFinish))

result2

True


AgentFinish(return_values={'output': "Olá! I'm Wesley Safadão, your contact management assistant. How can I help you today? I can verify, create, delete, or edit your contacts. Just let me know what you need!"}, log="Olá! I'm Wesley Safadão, your contact management assistant. How can I help you today? I can verify, create, delete, or edit your contacts. Just let me know what you need!")

I can now modify the route function to take in `AgentActionMessageLog` object instead of a dictionary.

In [ ]:
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result,AgentFinish):
        return print("No tools selected")
    else:
        tools = {
            "check_contact": check_contact,
            "add_contact": add_contact,
            "remove_contact": remove_contact,
            "edit_contact": edit_contact,
        }
        return tools[result[0].tool].run(result[0].tool_input)

Now test it with a simple call

In [ ]:
chain_route = prompt | chatmodel | ToolsAgentOutputParser() | route  # replacing ToolsOutputParser()

observation1 = chain_route.invoke({
    "input": "do I have a contact named Ronaldo?",
    "agent_scratchpad": []
})

observation1

{'success': True,
 'message': {'id': 6,
  'name': 'Ronaldo',
  'nickname': 'R9',
  'phone_number': '+15575867367',
  'email': 'fifathebest@uol.com'}}

📓 Note it gives the same as calling the function directly

In [ ]:
observation2 = check_contact({"name":"Ronaldo"})

observation2

{'success': True,
 'message': {'id': 6,
  'name': 'Ronaldo',
  'nickname': 'R9',
  'phone_number': '+15575867367',
  'email': 'fifathebest@uol.com'}}

✅ The `langchain.agents.format_scratchpad` function is part of the LangChain framework, which is designed to facilitate the development of applications using large language models (LLMs). Specifically, this function is used to format the intermediate steps taken by an agent during its operation, allowing for better tracking and management of the agent's thought process and actions.
Here this function is replaced by it equivalent `format_to_tool_messages()`

✅ The primary purpose of `format_scratchpad` is to convert the intermediate steps of an agent's decision-making process into a structured format that can be easily understood and utilized. This is particularly useful in scenarios where an agent needs to maintain context or reference previous actions and observations.

▶ The function takes a list of tuples as input, where each tuple consists of an AgentAction and its corresponding output (observation).

▶ These intermediate steps represent the actions the agent has taken and the results of those actions.

In [ ]:
# Let me test the usage and output of format_to_tool_messages() which was found to be an alternative to format_to_openai_functions()

from langchain.agents.format_scratchpad.tools import format_to_tool_messages

format_to_tool_messages([(result, observation2), ])

[AIMessage(content=[{'text': 'Let me check if you have a contact named Ronaldo in your database.', 'type': 'text'}, {'id': 'toolu_01KE8FAySN3z8Qft7cRzmgLX', 'input': {'name': 'Ronaldo'}, 'name': 'check_contact', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01DQXiceo7XuHPffU3asGNAL', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1614, 'output_tokens': 72, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--f740ec9f-78d1-47e1-b7f2-c86423a3e520-0', tool_calls=[{'name': 'check_contact', 'args': {'name': 'Ronaldo'}, 'id': 'toolu_01KE8FAySN3z8Qft7cRzmgLX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1614, 'output_tokens': 72, 'total_tokens': 1686, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}}),
 ToolMessage(content='{"success": true, "message": {"id": 6, "name": "Ronaldo", "nic

⚡ Now I can incorporate it into my agent 🤖 adapting from the function `route()`

In [ ]:
# adapted from route()
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_tool_messages(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        else:
          result_ = result[0]
          tool = {
            "check_contact": check_contact,
            "add_contact": add_contact,
            "remove_contact": remove_contact,
            "edit_contact": edit_contact,
          }[result_.tool]
          observation = tool.run(result_.tool_input)
          intermediate_steps.append((result_, observation))

In [ ]:
run_agent("do I have a contact named Ronaldo?")

AgentFinish(return_values={'output': 'Yes, you have a contact named Ronaldo! Here are the details:\n- Name: Ronaldo\n- Nickname: R9\n- Phone: +15575867367\n- Email: fifathebest@uol.com'}, log='Yes, you have a contact named Ronaldo! Here are the details:\n- Name: Ronaldo\n- Nickname: R9\n- Phone: +15575867367\n- Email: fifathebest@uol.com')

---


# ❗ As an alternative to the function **run_agent()**, I can use **AgentExecutor** 🏃

✅ In the following we use LangChain's `RunnablePassthrough` and a custom agent chain to process user input, execute tools, and return results.

▶ RunnablePassthrough is a utility in LangChain that passes inputs unchanged or adds additional keys to the output. It is often used in chains to forward data without modifying it.

▶ In this code, it is used with the `.assign()` method to add a new key `(agent_scratchpad)` to the input dictionary. It Merges the Dict input with the output produced by the mapping argument.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_tool_messages(x["intermediate_steps"])
) | chain

# chain = prompt | chatmodel | ToolsAgentOutputParser()

In [ ]:
# Assuming check_contact, add_contact, remove_contact, and edit_contact are the functions
# decorated with @tool as defined earlier in the notebook.
# The tools_box list for AgentExecutor should contain the tool objects, not just the functions nor the json.
# Recreate the list of tools using the decorated functions which are already Tool objects.

tools_box = [check_contact, add_contact, remove_contact, edit_contact]
tools = tools_box

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "do I have a contact named Ronaldo?"})



> Entering new AgentExecutor chain...

Invoking: `check_contact` with `{'name': 'Ronaldo'}`
responded: [{'text': 'Let me check if you have a contact named Ronaldo in your database.', 'type': 'text', 'index': 0}, {'id': 'toolu_017PjGKthuf1VWJ4AyjUe4hP', 'input': {}, 'name': 'check_contact', 'type': 'tool_use', 'index': 1, 'partial_json': '{"name": "Ronaldo"}'}]

{'success': True, 'message': {'id': 6, 'name': 'Ronaldo', 'nickname': 'R9', 'phone_number': '+15575867367', 'email': 'fifathebest@uol.com'}}[{'text': 'Yes, you have a contact named Ronaldo! Here are the details:\n- Name: Ronaldo\n- Nickname: R9\n- Phone: +15575867367\n- Email: fifathebest@uol.com', 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'do I have a contact named Ronaldo?',
 'output': [{'text': 'Yes, you have a contact named Ronaldo! Here are the details:\n- Name: Ronaldo\n- Nickname: R9\n- Phone: +15575867367\n- Email: fifathebest@uol.com',
   'type': 'text',
   'index': 0}]}

Let us test the memory capability 🧠

In [ ]:
agent_executor.invoke({"input": "Hi, my name is Yago Pikachu. What is your name?"})



> Entering new AgentExecutor chain...
[{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your helpful assistant for managing contacts. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll assist you right away!", 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'Hi, my name is Yago Pikachu. What is your name?',
 'output': [{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your helpful assistant for managing contacts. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll assist you right away!",
   'type': 'text',
   'index': 0}]}

In [ ]:
agent_executor.invoke({"input": "what is my name?"})



> Entering new AgentExecutor chain...
[{'text': "I don't have access to your personal information, including your name, unless you've shared it with me during our conversation. If you'd like, you can tell me your name, and I can use it when addressing you. How can I help you with managing your contacts today?", 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'what is my name?',
 'output': [{'text': "I don't have access to your personal information, including your name, unless you've shared it with me during our conversation. If you'd like, you can tell me your name, and I can use it when addressing you. How can I help you with managing your contacts today?",
   'type': 'text',
   'index': 0}]}

## Clearly the agent 🤖 is not capable to memorize things 🧠. I have to implement it with an AgentExecutor.

---

To add a memory buffer is pretty easy. Just increment the `ChatPromptTemplate` with message placeholder, deffine the `ConversationBufferMemory`, and add the memory object to the `AgentExecutor`

In [ ]:
from langchain_anthropic.output_parsers import ToolsOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# adapt the prompt to store message history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant named Wesley Safadão. Your task is to manage contact information for the user such as verify, create, delete, or edit a contact informations like contacts name, phone number, email, etc. Keep your answers short."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


In [ ]:
# Use the ConversationBufferMemory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


<ipython-input-65-4839071d5871>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [ ]:
# check the comments about RunnablePassthrough from before
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_tool_messages(x["intermediate_steps"])
) | prompt | chatmodel | ToolsAgentOutputParser()

In [ ]:
# and add memory to the agent executor.
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.invoke({"input": "Hi, my name is Yago Pikachu. What is your name?"})



> Entering new AgentExecutor chain...
[{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your contact management assistant. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll take care of it for you!", 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'Hi, my name is Yago Pikachu. What is your name?',
 'chat_history': [HumanMessage(content='Hi, my name is Yago Pikachu. What is your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your contact management assistant. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll take care of it for you!", 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={})],
 'output': [{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your contact management assistant. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll take care of it for you!",
   'type': 'text',
   'index': 0}]}

In [ ]:
agent_executor.invoke({"input": "what is my name?"})



> Entering new AgentExecutor chain...
[{'text': 'Your name is Yago Pikachu, as you mentioned earlier. How can I help you manage your contacts today?', 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'what is my name?',
 'chat_history': [HumanMessage(content='Hi, my name is Yago Pikachu. What is your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your contact management assistant. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll take care of it for you!", 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': 'Your name is Yago Pikachu, as you mentioned earlier. How can I help you manage your contacts today?', 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={})],
 'output': [{'text': 'Your name is Yago Pikachu, as you mentioned earlier. How can I help you manage your contacts today?',
   'type': 'text',
   'index': 0}]}

In [ ]:
agent_executor.invoke({"input": "do I have a contact named Ronaldo?"})



> Entering new AgentExecutor chain...

Invoking: `check_contact` with `{'name': 'Ronaldo'}`
responded: [{'text': 'Let me check if you have a contact named Ronaldo.', 'type': 'text', 'index': 0}, {'id': 'toolu_018AU3Wpd84EBodMhwHS3zbK', 'input': {}, 'name': 'check_contact', 'type': 'tool_use', 'index': 1, 'partial_json': '{"name": "Ronaldo"}'}]

{'success': True, 'message': {'id': 6, 'name': 'Ronaldo', 'nickname': 'R9', 'phone_number': '+15575867367', 'email': 'fifathebest@uol.com'}}[{'text': 'Yes, you have a contact named Ronaldo! Here are the details:\n- Name: Ronaldo\n- Nickname: R9\n- Phone: +15575867367\n- Email: fifathebest@uol.com', 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'do I have a contact named Ronaldo?',
 'chat_history': [HumanMessage(content='Hi, my name is Yago Pikachu. What is your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your contact management assistant. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll take care of it for you!", 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': 'Your name is Yago Pikachu, as you mentioned earlier. How can I help you manage your contacts today?', 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={}),
  HumanMessage(content='do I have a contact named Ronaldo?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': 'Yes, you have a contact named Ronaldo! He

In [ ]:
agent_executor.invoke({"input": "Create a new contact named José Celso aka zé, with phone number +4538770890, email zecelso@uol.com"})



> Entering new AgentExecutor chain...

Invoking: `add_contact` with `{'name': 'José Celso', 'nickname': 'zé', 'phone_number': '+4538770890', 'email': 'zecelso@uol.com'}`
responded: [{'id': 'toolu_01QetA9tHC2JeE2Gp8waevwu', 'input': {}, 'name': 'add_contact', 'type': 'tool_use', 'index': 0, 'partial_json': '{"name": "José Celso", "nickname": "zé", "phone_number": "+4538770890", "email": "zecelso@uol.com"}'}]

{'success': True, 'message': 'A new contact was added successfully.'}[{'text': 'Pronto! I\'ve added José Celso to your contacts with the nickname "zé", phone number +4538770890, and email zecelso@uol.com. The contact was created successfully!', 'type': 'text', 'index': 0}]

> Finished chain.


{'input': 'Create a new contact named José Celso aka zé, with phone number +4538770890, email zecelso@uol.com',
 'chat_history': [HumanMessage(content='Hi, my name is Yago Pikachu. What is your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': "Olá, Yago Pikachu! I'm Wesley Safadão, your contact management assistant. I can help you verify, create, delete, or edit contact information. Just let me know what you'd like to do with your contacts, and I'll take care of it for you!", 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'text': 'Your name is Yago Pikachu, as you mentioned earlier. How can I help you manage your contacts today?', 'type': 'text', 'index': 0}], additional_kwargs={}, response_metadata={}),
  HumanMessage(content='do I have a contact named Ronaldo?', additional_kwargs={}, response_metadata={}),
  AIMess

In [ ]:
print_contacts()

   id             name nickname  phone_number                 email
0   1  Ana Maria Braga   MaisVC  +12124567890    anamaria@globo.com
1   2     Fausto Silva  Faustao   +4530770800      fausto@globo.com
2   4  Thammy Gretchen  Carluxa  +13105867367     tammy@example.com
3   6          Ronaldo       R9  +15575867367   fifathebest@uol.com
4   8     Rony Rustico     Rony   +4538711490  rony.rustico@uol.com
5   9       José Celso       zé   +4538770890       zecelso@uol.com


### Create a chatbot 🤖

In [ ]:
%%capture
!pip install jupyter_bokeh
!pip install panel

In [ ]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class WesleySafa(param.Parameterized):
    """ Conversational Agent Chatbot. Takes tools_box as input. """
    def __init__(self, tools, **params):
        super(WesleySafa, self).__init__( **params)
        self.panels = []
        self.functions = [convert_to_anthropic_tool(f) for f in tools]
        self.model = ChatAnthropic(model="claude-3-7-sonnet-20250219",
                                   temperature=0).bind(tools=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful personal assitant called Wesley Safadão. Keep your answers short and friendly."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_tool_messages(x["intermediate_steps"])
        ) | self.prompt | self.model | ToolsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'][0]["text"]
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)
        # return self.answer


    def clr_history(self,count=0):
        self.chat_history = []
        return

In [ ]:
# Use this if want to return only self.answer

# cb = WesleySafa(tools)

# conversation = cb.convchain("Hello!")

# conversation

In [ ]:
print_contacts()

   id              name nickname  phone_number                 email
0   1   Ana Maria Braga   MaisVC  +12124567890    anamaria@globo.com
1   2      Fausto Silva  Faustao   +4530770800      fausto@globo.com
2   4   Thammy Gretchen  Carluxa  +13105867367     tammy@example.com
3   5  Carlos Bolsonaro  Carluxo   +4538770890                  None
4   6           Ronaldo       R9  +15575867367   fifathebest@uol.com
5   8      Rony Rustico     Rony   +4538711490  rony.rustico@uol.com


In [ ]:
cb = WesleySafa(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True)
            [3] Divider()

In [ ]:
print_contacts()

   id             name nickname  phone_number                 email
0   1  Ana Maria Braga   MaisVC  +12124567890    anamaria@globo.com
1   2     Fausto Silva  Faustao   +4530770800      fausto@globo.com
2   4  Thammy Gretchen  Carluxa  +13105867367     tammy@example.com
3   6          Ronaldo       R9  +15575867367   fifathebest@uol.com
4   8     Rony Rustico     Rony   +4538711490  rony.rustico@uol.com
5   9       José Celso       zé   +4538770890       zecelso@uol.com
